# Histogram Model: Color Feature
----
Sergei Papulin (papulin.study@yandex.ru)

## Contents

- Loading Elements
- Image and its Histogram
- Basic Operations in Histogram Model
    - Set Operations
    - Logical Operations
- Analyzing Image using Histogram Model
- Image Retrieval by Expression
- Image Retrieval by Sample
- References

#### Load packages

In [ ]:
import sys
from plotly.subplots import make_subplots

sys.path.insert(0, "../../")

from lshist.utils import (E, convert_hist_to_all_values)

from utils import (
    load_hist_elements_from_images, 
    load_hist_elements_from_json, 
    convert2hist,
    hsl2rgb,
    get_rgb_colors,
    generate_image,
    show_histogram)

## Loading Elements


Stucture of a json file with descriptions of color elements:

```json
{
    "elements": [{
        "id": id,
        "h": [min, max],
        "s": [min, max],
        "b": [min, max]},...]
}
```

Parameters:

- `id`: identification of an element
- `h`: hue, interval. If you define interval from 230 to 10, it will be divided into two ones: from 230 to 240 and from 0 to 10
- `s`: saturation, interval
- `b`: brightness, interval


In [ ]:
ELEMENT_PATH = "elements.json"

color_elements = load_hist_elements_from_json(ELEMENT_PATH)
color_elements[:3]

In [ ]:
color_converter = get_rgb_colors(color_elements)
color_converter["e1"]

Show all colors:

In [ ]:
U = list(item["id"] for item in color_elements)
colors = ["rgb{}".format(color_converter[el]) for el in U]

In [ ]:
U_val_all = [1 for el in U]

In [ ]:
hist_list = [U_val_all]
titles = ["U (all elements)"]
names = ["U"]

fig = show_histogram(hist_list, U, colors, titles, names, main_title="Color Universal Elements")
fig.show()

$$E^G=\left\{E\vert E\subseteq U\right\}$$

Create high-level elements:

In [ ]:
E1 = E("e1+e2+e3+e4+e5+e6+e7+e8+e9+e10+e11+e12+e13+e14+e15+e16+e17+e18+e19+e20")  # green
E2 = E("e2+e3+e21+e22+e23+e24+e25+e26+e27+e28+e29+e30")  # yellow-green
E3 = E("e31+e32+e33+e34+e35+e36+e37+e38+e39+e40")  # red
E4 = E("e32+e35+e36+e39+e40")  # pink

Show elements:

In [ ]:
E1_el = E1.value.strip("()").split("+")
E1_val_all = [1 if el in E1_el else 0 for el in U]

E2_el = E2.value.strip("()").split("+")
E2_val_all = [1 if el in E2_el else 0 for el in U]

E3_el = E3.value.strip("()").split("+")
E3_val_all = [1 if el in E3_el else 0 for el in U]

E4_el = E4.value.strip("()").split("+")
E4_val_all = [1 if el in E4_el else 0 for el in U]

In [ ]:
hist_list = [U_val_all, E1_val_all, E2_val_all, E3_val_all, E4_val_all]
titles = ["U (all elements)", "E1 (green)", "E2 (yellow-green)", "E3 (red)", "E4 (pink)"]
names = ["U", "E1", "E2", "E3", "E4"]

fig = show_histogram(hist_list, U, colors, titles, names)
fig.show()

## Image and its Histogram

Generate an image composed from elements of the universal set:

In [ ]:
img = generate_image(U, color_converter, seed=1234)
img

Create a histogram from the image

In [ ]:
hist = convert2hist(img, color_elements, mode="json")
hist

Showing Image and its Histogram

In [ ]:
hist_elements = sorted(hist.hist_elements().items(), key=lambda x: int(x[0].strip("e")))

elements = [el[0] for el in hist_elements]
values = [el[1].value for el in hist_elements]
colors = ["rgb{}".format(color_converter[el]) for el in elements]

In [ ]:
colors_all = ["rgb{}".format(color_converter[el]) for el in U]
elements_all = convert_hist_to_all_values(U, hist)

In [ ]:
fig = make_subplots(rows=2, cols=2, specs=[[{}, {}],[{"colspan": 2}, None]], subplot_titles=("Image", "Histogram", "Full Histogram"))

fig.add_image(z=img, row=1, col=1, name="image")
fig.add_bar(x=elements, y=values, marker_color=colors, width=0.5, row=1, col=2, name="histogram")
fig.add_bar(x=U, y=elements_all, marker_color=colors_all, row=2, col=1, name="E1")

fig.update_xaxes(gridcolor='#bdbdbd', title="Elements", titlefont=dict(color="grey"), row=1, col=2)
fig.update_yaxes(gridcolor='#bdbdbd', title="Counts", titlefont=dict(color="grey"), row=1, col=2)
fig.update_xaxes(gridcolor='#bdbdbd', title="Elements", titlefont=dict(color="grey"), row=2, col=1)
fig.update_yaxes(gridcolor='#bdbdbd', title="Counts", titlefont=dict(color="grey"), row=2, col=1)

fig.update_layout(plot_bgcolor='#fefefe', showlegend=False, height=480, width=800, title_text="Initial Data")
fig.show()

Total number of elements in the image:

In [ ]:
len(hist)

Non-zero elements of the histogram:

In [ ]:
hist.elements()

In [ ]:
# TODO: total number of histogram elements

Sum of all histogram element values:

In [ ]:
hist.sum()

Get a value of the first histogram element:

In [ ]:
hist(hist.elements()[0]).sum()

Get a value of the `e1` element:

In [ ]:
hist("e1").sum()

or

In [ ]:
hist.hist_elements()["e1"].value

## Basic Operations in Histogram Model

In [ ]:
from lshist.executor import Parser, Evaluator
from lshist.histogram import operations

In [ ]:
parser = Parser()
evaluator = Evaluator(operations, hist)

$${H}^{N}=\left( H \mid H=\bigcup_{x\in U}{H \left(x \right)} /x, 0 \leq H \left( x \right) \leq 1, x \in U, \sum_{x\in U}H\left(x\right) = 1 \right)$$

$${H}^{G} = \left \lbrace H \mid H=\bigcup_{x\in E}{H \left(x \right) /x} ,0 \leq H \left(x \right) ≤ {H} ^ {N} \left(x \right) ,E⊆U,  \sum_{x\in E}H\left(x\right)≤1 \right \rbrace$$

$$\mathcal H=\left\langle H^G,\cap,\cup,\wedge,\dot\vee,\vee,\dot\backslash,\setminus,\overline{},\mathbf{0},\mathbf{1}\right\rangle$$

$$\mathbf{0}=H^0={\displaystyle \left\{H\mid H\in H^G,\sum_{x\in U}H\left(x\right)=0\right\}}$$

$$\mathbf{1}=H^1=H^N=\left\{H\mid H\in H^G,\sum_{x\in U}H\left(x\right)=1\right\}$$

#### Histograms of E1, E2, E3 and E4 given Image

Create a histogram of E1 given the image:

In [ ]:
E1_expr = parser.parse_string(E1.value)
HE1 = evaluator.eval(E1_expr)

print("Expression for E1:\n{}".format(E1.value))
print("\nThe parsed expressino for E1 in the postfix notation:\n{}".format(E1_expr))
print("\nHistogram of E1 given the image:\n{}".format(HE1.to_dict()))

Same for E2, E3 and E4:

In [ ]:
E2_expr = parser.parse_string(E2.value)
HE2 = evaluator.eval(E2_expr)

E3_expr = parser.parse_string(E3.value)
HE3 = evaluator.eval(E3_expr)

E4_expr = parser.parse_string(E4.value)
HE4 = evaluator.eval(E4_expr)

Show the histograms of E1, E2, E3 and E4 given the image:

In [ ]:
HE1_all = convert_hist_to_all_values(U, HE1)
HE2_all = convert_hist_to_all_values(U, HE2)
HE3_all = convert_hist_to_all_values(U, HE3)
HE4_all = convert_hist_to_all_values(U, HE4)

In [ ]:
hist_list = [HE1_all, HE2_all, HE3_all, HE4_all]
titles = ["E1 (green)", "E2 (yellow-green)", "E3 (red)", "E4 (pink)"]
names = ["E1", "E2", "E3", "E4"]

fig = show_histogram(hist_list, U, colors_all, titles, names)
fig.show()

### Set Operations

#### UNION

$$H_1\cup H_2=\left(H\mid H=\bigcup_{x\in U}H\left(x\right)/x,H\left(x\right)=\max\left(H_1\left(x\right),H_2\left(x\right)\right)\right)$$

In [ ]:
E_union = E1 + E2
E_union_expr = parser.parse_string(E_union.value)
HE_union = evaluator.eval(E_union_expr)

print("Expression for E_union:\n{}".format(E_union))
print("\nThe parsed expression for E_union in the postfix notation:\n{}".format(E_union_expr))
print("\nHistogram of E_union given the image:\n{}".format(HE_union.to_dict()))
print("\nValue of presence for E_union:\n{}".format(HE_union.sum()))

Show the histogram of E_union given the image:

In [ ]:
HE1_union_HE2_all = convert_hist_to_all_values(U, HE_union)

hist_list = [HE1_all, HE2_all, HE1_union_HE2_all]
titles = ["E1 (green)", "E2 (yellow-green)", "E1 + E2"]
names = ["E1", "E2", "E1+E2"]

fig = show_histogram(hist_list, U, colors_all, titles, names)
fig.show()

#### INTERSECTION

$$H_1\cap H_2=\left(H\mid H=\bigcup_{x\in U}H\left(x\right)/x,H\left(x\right)=\min\left(H_1\left(x\right),H_2\left(x\right)\right)\right)$$

In [ ]:
E_intersect = E1 * E2  # or E1.Intersection(E2)
E_intersect_expr = parser.parse_string(E_intersect.value)
HE_intersect = evaluator.eval(E_intersect_expr)

print("Expression for E_intersect:\n{}".format(E_intersect))
print("\nThe parsed expression for E_intersect in the postfix notation:\n{}".format(E_intersect_expr))
print("\nHistogram of E_intersect given the image:\n{}".format(HE_intersect.to_dict()))
print("\nValue of presence for E_intersect:\n{}".format(HE_intersect.sum()))

Show the histogram of E_intercept given the image:

In [ ]:
HE1_intersect_HE2_all = convert_hist_to_all_values(U, HE_intersect)

hist_list = [HE1_all, HE2_all, HE1_intersect_HE2_all]
titles = ["E1 (green)", "E2 (yellow-green)", "E1 * E2"]
names = ["E1", "E2", "E1*E2"]

fig = show_histogram(hist_list, U, colors_all, titles, names)
fig.show()

#### SUBSTRACTION or EXCEPTION

$$H_1\backslash H_2=\left(H\mid H=\bigcup_{x\in U}H\left(x\right)/x,H\left(x\right)=\left\{\begin{array}{l}0\;if\;H_2\left(x\right)>0,\\H_1\left(x\right)\;otherwise\end{array}\right.\right)$$

In [ ]:
E_sub = E1 - E2  # or E1.Sub(E2)
E_sub_expr = parser.parse_string(E_sub.value)
HE_sub = evaluator.eval(E_sub_expr)

print("Expression for E_sub:\n{}".format(E_sub))
print("\nThe parsed expression for E_sub in the postfix notation:\n{}".format(E_sub_expr))
print("\nHistogram of E_sub given the image:\n{}".format(HE_sub.to_dict()))
print("\nValue of presence for E_sub:\n{}".format(HE_sub.sum()))

Show the histogram of E_sub given the image:

In [ ]:
HE1_sub_HE2_all = convert_hist_to_all_values(U, HE_sub)

hist_list = [HE1_all, HE2_all, HE1_sub_HE2_all]
titles = ["E1 (green)", "E2 (yellow-green)", "E1 - E2"]
names = ["E1", "E2", "E1-E2"]

fig = show_histogram(hist_list, U, colors_all, titles, names)
fig.show()

### Logical Operations

#### AND

$$H_1\wedge H_2=\left\{\begin{array}{l}H_1\;if\;\sum_{x\in U}H_1\left(x\right)<\sum_{x\in U}H_2\left(x\right),\\H_2\;otherwise\end{array}\right.$$

In [ ]:
E_and = E1 & E2  # or E1.And(E2)
E_and_expr = parser.parse_string(E_and.value)
HE_and = evaluator.eval(E_and_expr)

print("Expression for E_and:\n{}".format(E_and))
print("\nThe parsed expression for E_and in the postfix notation:\n{}".format(E_and_expr))
print("\nHistogram of E_and given the image:\n{}".format(HE_and.to_dict()))
print("\nValue of presence for E_and:\n{}".format(HE_and.sum()))

Show the histogram of E_and given the image:

In [ ]:
HE1_and_HE2_all = convert_hist_to_all_values(U, HE_and)
hist_list = [HE1_all, HE2_all, HE1_and_HE2_all]
titles = ["E1 (green)", "E2 (yellow-green)", "E1 <b>AND</b> E2"]
names = ["E1", "E2", "E1&E2"]

fig = show_histogram(hist_list, U, colors_all, titles, names)
fig.show()

#### OR

$$H_1\vee H_2=\left(H\mid H=\bigcup_{x\in U}H\left(x\right)/x,H\left(x\right)=\max\left(H_1\left(x\right),H_2\left(x\right)\right)\right)$$

In [ ]:
E_or = E1 | E2  # or E1.Or(E2)
E_or_expr = parser.parse_string(E_or.value)
HE_or = evaluator.eval(E_or_expr)

print("Expression for E_or:\n{}".format(E_or))
print("\nThe parsed expression for E_or in the postfix notation:\n{}".format(E_or_expr))
print("\nHistogram of E_or given the image:\n{}".format(HE_or.to_dict()))
print("\nValue of presence for E_or:\n{}".format(HE_or.sum()))

Show the histogram of E_or given the image:

In [ ]:
HE1_or_HE2_all = convert_hist_to_all_values(U, HE_or)
hist_list = [HE1_all, HE2_all, HE1_or_HE2_all]
titles = ["E1 (green)", "E2 (yellow-green)", "E1 <b>OR</b> E2"]
names = ["E1", "E2", "E1|E2"]

fig = show_histogram(hist_list, U, colors_all, titles, names)
fig.show()

#### XOR

$$H_1\dot\vee H_2=\left\{\begin{array}{l}H_1\;if\;\sum_{x\in U}H_1\left(x\right)>\sum_{x\in U}H_2\left(x\right),\\H_2\;otherwise\end{array}\right.$$

In [ ]:
E_xor = E1 ^ E2  # or E1.Xor(E2)
E_xor_expr = parser.parse_string(E_xor.value)
HE_xor = evaluator.eval(E_xor_expr)

print("Expression for E_xor:\n{}".format(E_xor))
print("\nThe parsed expression for E_xor in the postfix notation:\n{}".format(E_xor_expr))
print("\nHistogram of E_xor given the image:\n{}".format(HE_xor.to_dict()))
print("\nValue of presence for E_xor:\n{}".format(HE_xor.sum()))

Show the histogram of E_xor given the image:

In [ ]:
HE1_xor_HE2_all = convert_hist_to_all_values(U, HE_xor)
hist_list = [HE1_all, HE2_all, HE1_xor_HE2_all]
titles = ["E1 (green)", "E2 (yellow-green)", "E1 <b>XOR</b> E2"]
names = ["E1", "E2", "E1^E2"]

fig = show_histogram(hist_list, U, colors_all, titles, names)
fig.show()

#### XSUBSTRACTION

$$H_1\dot\backslash H_2=\left\{\begin{array}{l}0\;if\;\sum_{x\in U}H_2\left(x\right)>0,\\H_1\;otherwise\end{array}\right.$$

In [ ]:
E_xsub = E1.Xsub(E2)
E_xsub_expr = parser.parse_string(E_xsub.value)
HE_xsub = evaluator.eval(E_xsub_expr)

print("Expression for E_xsub:\n{}".format(E_xsub))
print("\nThe parsed expression for E_xsub in the postfix notation:\n{}".format(E_xsub_expr))
print("\nHistogram of E_xsub given the image:\n{}".format(HE_xsub.to_dict()))
print("\nValue of presence for E_xsub:\n{}".format(HE_xsub.sum()))

Show the histogram of E_xsub given the image:

In [ ]:
HE1_xsub_HE2_all = convert_hist_to_all_values(U, HE_xsub)
hist_list = [HE1_all, HE2_all, HE1_xsub_HE2_all]
titles = ["E1 (green)", "E2 (yellow-green)", "E1 <b>XSUBSTRACT</b> E2"]
names = ["E1", "E2", "E1#/E2"]

fig = show_histogram(hist_list, U, colors_all, titles, names)
fig.show()

In [ ]:
E_xsub = E1.Xsub(E("e17+e18+e19"))
E_xsub_expr = parser.parse_string(E_xsub.value)
HE_xsub = evaluator.eval(E_xsub_expr)

print("Expression for E_xsub:\n{}".format(E_xsub))
print("\nThe parsed expression for E_xsub in the postfix notation:\n{}".format(E_xsub_expr))
print("\nHistogram of E_xsub given the image:\n{}".format(HE_xsub.to_dict()))
print("\nValue of presence for E_xsub:\n{}".format(HE_xsub.sum()))

Show the histogram of E_xsub given the image:

In [ ]:
E5_expr = parser.parse_string(E("e17+e18+e19").value)
HE5 = evaluator.eval(E5_expr)
HE5_all = convert_hist_to_all_values(U, HE5)

HE1_xsub_HE2_all = convert_hist_to_all_values(U, HE_xsub)
hist_list = [HE1_all, HE5_all, HE1_xsub_HE2_all]
titles = ["E1 (green)", "E5", "E1 <b>XSUBSTRACT</b> E5"]
names = ["E1", "E5", "E1#/E5"]

fig = show_histogram(hist_list, U, colors_all, titles, names)
fig.show()

## Analyzing Image using Histogram Model

Set a query:

In [ ]:
query = (E1 + E2) & E3  # (green or yellow-green) and red

Compute a histogram of the query given the image:

In [ ]:
expr = parser.parse_string(query.value)
HE = evaluator.eval(expr)

In [ ]:
print("\nHistogram of guery given the image:\n{}".format(HE.to_dict()))
print("\nValue of presence:\n{}".format(HE.sum()))

## Image Retrieval by Expression

Generate images:

In [ ]:
imgs = [generate_image(U, color_converter, seed=i) for i in range(100)]

Create histograms for the images:

In [ ]:
hists = [convert2hist(img, color_elements, mode="json") for img in imgs]

Compose a query to get a subset of relevant images:

In [ ]:
query = (E1 + E2) & E3  # (green or yellow-green) and red
expr = parser.parse_string(query.value)

Calculate histograms of the query for the images:

In [ ]:
HEs = [evaluator.eval(expr, hist) for hist in hists]

Rank the images based on the value of presence:

In [ ]:
img_rank = sorted([(indx, HE.sum()) for indx, HE in enumerate(HEs)], key=lambda x: -x[1])
print("Top 5 images:", img_rank[:5])
print("Last 5 images:", img_rank[-5:])

Image that most corresponds to the query:

In [ ]:
imgs[img_rank[0][0]]

Image that least corresponds to the query:

In [ ]:
imgs[img_rank[len(imgs)-1][0]]

## Image Retrieval by Sample

Generate a sample image:

In [ ]:
img_sample = generate_image(E3_el, color_converter, seed=0)
img_sample

Create the histogram of the image:

In [ ]:
hist_sample = convert2hist(img_sample, color_elements, mode="json")

Rank the images according to their similarity:

In [ ]:
img_sample_rank = sorted([(indx, (hist_sample * hist).sum()) for indx, hist in enumerate(hists)], 
                         key=lambda x: -x[1])
print("Top 5 images:", img_sample_rank[:5])
print("Last 5 images:", img_sample_rank[-5:])

Show the most similar image:

In [ ]:
imgs[img_sample_rank[0][0]]

Show the least similar image:

In [ ]:
imgs[img_sample_rank[len(imgs)-1][0]]

## References